In [56]:
import sys
from pathlib import Path
import importlib
import ipynbname 
import pandas as pd

code_path = ipynbname.path().parent.parent
# Ajouter le dossier scripts au path
scripts_path = code_path  / "scripts"
sys.path.append(str(scripts_path.resolve()))

import data_utils  # importe le module une première fois

# Après avoir modifié data_utils.py
importlib.reload(data_utils)

base_path = code_path.parent
# Maintenant tu peux accéder aux fonctions mises à jour
from data_utils import import_data_raw, import_data_sig, melt_long_format, clean_year_column, save_long_dataframe, concat_intermediate_files


## Importation et formatage

In [122]:
#Visualisation du fichier
filename="GCB_FFConsumption_Countries.csv"
filepath= base_path/ "Data" / 'data_raw' / filename

df = pd.read_csv(filepath, dtype=str,header=1)
df.head()

,Unnamed: 0,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
# Définir le chemin
filename = "National_Fossil_Carbon_Emissions_2024v1.0-1.xlsx"
filepath = base_path / "Data" / 'GHG Datasets' / 'Global Carbon Budget' / filename

# Charger le fichier Excel
xls = pd.ExcelFile(filepath)

# Afficher les noms des feuilles
print("Feuilles dans le fichier :", xls.sheet_names)

# Lire la feuille souhaitée
df = pd.read_excel(filepath, sheet_name=xls.sheet_names[1], dtype=str, header=11)

# Renommer la première colonne en 'Year'
df.rename(columns={df.columns[0]: "Year"}, inplace=True)

# Vérifier
df.head(10)


Feuilles dans le fichier : ['Summary', 'Territorial Emissions', 'Consumption Emissions', 'Emissions Transfers', 'Regions']


,Year,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,...,Asia,Central America,Europe,Middle East,North America,Oceania,South America,Bunkers,Statistical Difference,World
0,1850,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,6.8521138750599e-05,...,0.00401202150994847,0,48.2464969314514,0,5.418,0.0305820009434216,0.0256893,0,-1.4210854715202e-14,53.7247802539048
1,1851,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,...,0,0,47.45060918332,0,6.742,0.0290334908417313,0.0301441,0,-7.105427357601e-15,54.2517867741618
2,1852,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,...,0,0,49.24473830895,0,7.335,0.0289450290888339,0.0493654,0,0,56.6580487380389
3,1853,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,...,0,0,50.9561893611538,0,8.26,0.0415723001759676,0.0318038,0,-7.105427357601e-15,59.2895654613298
4,1854,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,...,0,0,60.4710333066774,0,9.084,0.0500891057352644,0.0014707,0,1.4210854715202e-14,69.6065931124127
5,1855,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.000207773775566332,...,0.0121654845785534,0,60.4634048089304,0,10.456,0.0588639963115096,0.0411845,0,-1.4210854715202e-14,71.0316187898205
6,1856,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0045444,0,...,0,0.1049454,64.6237705939835,0,10.9781749,0.0815737600990279,0.1519357,0,0,75.9404003540825
7,1857,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0111237,0,...,0,0.0984529,65.0462936367495,0,11.2659992,0.0903658277146707,0.2404409,0,-1.4210854715202e-14,76.7415524644642
8,1858,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0144018,0.000291767429518679,...,0.124749367485013,0.0923615,65.9174383789256,0,11.4432252,0.0929264948628625,0.2628115,0,0,77.9335124412735
9,1859,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0092316,0.000309450304034963,...,0.191831889530606,0.0866474,69.5046705677605,0,12.4571777,0.132354671096029,0.20706,0,0,82.5797422283872


In [127]:
#Passage en format long
source="GCB"
unit="tC" 
facteur_conversion=1e6*0.272727  #3.66666 from C to C02, 0.272727 from CO2 to C 
indicator="Fossil Fuel Consumption"
df_long = melt_long_format(df, id_vars=['Year'], var_name='Country', value_name='Value', source=source,unit=unit, indicator=indicator)

df_long["Value"] = pd.to_numeric(df_long["Value"], errors="coerce") * facteur_conversion


df_long.head()



,Year,Country,Value,Unit,Indicator,Source
0,1850,Afghanistan,NaN,tC,Fossil Fuel Consumption,GCB
1,1851,Afghanistan,NaN,tC,Fossil Fuel Consumption,GCB
2,1852,Afghanistan,NaN,tC,Fossil Fuel Consumption,GCB
3,1853,Afghanistan,NaN,tC,Fossil Fuel Consumption,GCB
4,1854,Afghanistan,NaN,tC,Fossil Fuel Consumption,GCB


## SIG ET NOM DE PAYS

In [28]:
# Importation de fichier SIG
gdf_world=import_data_sig(base_path,'world-administrative-boundaries.geojson')
#gdf_world[['iso_3166_1_alpha_2_codes', 'name']]

gdf_world.head()

,geo_point_2d,iso3,status,color_code,name,continent,region,iso_3166_1_alpha_2_codes,french_short,geometry
0,"{ ""lon"": -16.984917430414384, ""lat"": 32.747916...",None,PT Territory,PRT,Madeira Islands,Europe,Southern Europe,None,Madeira Islands,"POLYGON ((-17.1025 32.82333, -17.05306 32.8094..."
1,"{ ""lon"": 33.743791080217562, ""lat"": 21.8927401...",None,Adm. by EGY,EGY,Ma'tan al-Sarra,Africa,Northern Africa,None,Ma'tan al-Sarra,"POLYGON ((33.25104 21.99977, 34.15064 21.99603..."
2,"{ ""lon"": 9.5613358449883421, ""lat"": 34.1108585...",TUN,Member State,TUN,Tunisia,Africa,Northern Africa,TN,Tunisie,"MULTIPOLYGON (((10.99361 33.75, 10.93778 33.72..."
3,"{ ""lon"": 43.77213543247138, ""lat"": 33.04802449...",IRQ,Member State,IRQ,Iraq,Asia,Western Asia,IQ,Iraq,"POLYGON ((44.78734 37.14971, 44.76617 37.11228..."
4,"{ ""lon"": -6.3178452255610269, ""lat"": 31.883624...",MAR,Member State,MAR,Morocco,Africa,Northern Africa,MA,Maroc,"POLYGON ((-2.94694 35.32916, -2.96618 35.31663..."


In [61]:
# Importation de fichier SIG
gdf_world=import_data_sig(base_path,'world.geojson')
#gdf_world[['iso_3166_1_alpha_2_codes', 'name']]

gdf_world.head()

,geo_point_2d,iso3,status,color_code,name,continent,region,Country_code,french_short,Country,geometry
0,"{ ""lon"": -16.984917430414384, ""lat"": 32.747916...",None,PT Territory,PRT,Madeira Islands,Europe,Southern Europe,None,Madeira Islands,None,"POLYGON ((-17.1025 32.82333, -17.05306 32.8094..."
1,"{ ""lon"": 33.743791080217562, ""lat"": 21.8927401...",None,Adm. by EGY,EGY,Ma'tan al-Sarra,Africa,Northern Africa,None,Ma'tan al-Sarra,None,"POLYGON ((33.25104 21.99977, 34.15064 21.99603..."
2,"{ ""lon"": 9.5613358449883421, ""lat"": 34.1108585...",TUN,Member State,TUN,Tunisia,Africa,Northern Africa,TN,Tunisie,Tunisia,"MULTIPOLYGON (((10.99361 33.75, 10.93778 33.72..."
3,"{ ""lon"": 43.77213543247138, ""lat"": 33.04802449...",IRQ,Member State,IRQ,Iraq,Asia,Western Asia,IQ,Iraq,Iraq,"POLYGON ((44.78734 37.14971, 44.76617 37.11228..."
4,"{ ""lon"": -6.3178452255610269, ""lat"": 31.883624...",MAR,Member State,MAR,Morocco,Africa,Northern Africa,MA,Maroc,Morocco,"POLYGON ((-2.94694 35.32916, -2.96618 35.31663..."


### OPTIONAL

In [62]:
import pandas as pd

# Fusion des deux tables
df_merged = df_long.merge(
    gdf_world[['iso_3166_1_alpha_2_codes', 'name']],
    left_on='Country',
    right_on='name',
    how='outer',  # on garde tout pour pouvoir identifier les non-correspondances
    indicator=True
)

# --- Identifier les correspondances et les non-correspondances ---
# Country sans équivalent dans gdf_world['name']
country_sans_name = df_merged[df_merged['_merge'] == 'left_only'][['Country']].drop_duplicates()

# name sans équivalent dans df_final['Country']
name_sans_country = df_merged[df_merged['_merge'] == 'right_only'][['name']].drop_duplicates()

# Comptage
print(f"Nombre de 'Country' sans équivalent : {len(country_sans_name)}")
print(f"Nombre de 'name' sans équivalent : {len(name_sans_country)}")

# Optionnel : garder seulement les ligne


KeyError: "['iso_3166_1_alpha_2_codes'] not in index"

In [63]:
import pandas as pd
from difflib import get_close_matches

# --- Préparation ---
countries = df_long['Country'].dropna().unique()
gdf_world['Country'] = None  # colonne à créer/remplir

from difflib import get_close_matches

# --- Fonction utilitaire pour trouver les meilleures correspondances ---
def find_best_match(target, candidates, cutoff=0.8):
    """Retourne les meilleures correspondances au-dessus du seuil de similarité."""
    return get_close_matches(target, candidates, n=5, cutoff=cutoff)

# --- Itération sur les lignes de gdf_world ---
for idx, row in gdf_world.iterrows():
    name = str(row['name'])
    french = str(row.get('french_short', '') or '')
    candidates = list(countries)

    # --- 1️⃣ Correspondance exacte ---
    exact_match = next(
        (c for c in candidates if c.lower() == name.lower() or c.lower() == french.lower()), 
        None
    )
    if exact_match:
        gdf_world.at[idx, 'Country'] = exact_match
        continue

    # --- 2️⃣ Correspondance par inclusion (dans un sens ou l’autre) ---
    inclusions = [
        c for c in candidates
        if name.lower() in c.lower() or c.lower() in name.lower()
        or french.lower() in c.lower() or c.lower() in french.lower()
    ]
    if inclusions:
        print(f"\n🌍 Nom : {name} (FR : {french})")
        print("→ Correspondances par inclusion :")
        for i, c in enumerate(inclusions):
            print(f"  [{i}] {c}")
        choix = input("Sélectionner le numéro de la correspondance à valider (Entrée pour ignorer) : ")

        if choix.isdigit() and int(choix) < len(inclusions):
            gdf_world.at[idx, 'Country'] = inclusions[int(choix)]
            print(f"✅ Correspondance validée (inclusion) : {name} → {inclusions[int(choix)]}")
            continue
        else:
            print(f"❌ Aucune correspondance validée pour {name}")

    # --- 3️⃣ Correspondance floue (difflib) ---
    matches_name = find_best_match(name, candidates)
    matches_fr = find_best_match(french, candidates) if french else []
    all_matches = list(dict.fromkeys(matches_name + matches_fr))  # fusion sans doublons

    if all_matches:
        print(f"\n🌍 Nom : {name} (FR : {french})")
        print("→ Correspondances floues potentielles dans df_merged['Country'] :")
        for i, c in enumerate(all_matches):
            print(f"  [{i}] {c}")
        choix = input("Sélectionner le numéro de la correspondance à valider (Entrée pour ignorer) : ")

        if choix.isdigit() and int(choix) < len(all_matches):
            gdf_world.at[idx, 'Country'] = all_matches[int(choix)]
            print(f"✅ Correspondance validée (floue) : {name} → {all_matches[int(choix)]}")
        else:
            print(f"❌ Aucune correspondance validée pour {name}")

print("\n✅ Mise à jour terminée.")




🌍 Nom : Netherlands Antilles (FR : Netherlands Antilles)
→ Correspondances par inclusion :
  [0] Netherlands


Sélectionner le numéro de la correspondance à valider (Entrée pour ignorer) :  


❌ Aucune correspondance validée pour Netherlands Antilles

🌍 Nom : British Indian Ocean Territory (FR : British Indian Ocean Territory)
→ Correspondances par inclusion :
  [0] India


Sélectionner le numéro de la correspondance à valider (Entrée pour ignorer) :  


❌ Aucune correspondance validée pour British Indian Ocean Territory

🌍 Nom : Antigua & Barbuda (FR : Antigua-et-Barbuda)
→ Correspondances floues potentielles dans df_merged['Country'] :
  [0] Antigua and Barbuda


...

❌ Aucune correspondance validée pour American Samoa

🌍 Nom : South Georgia & the South Sandwich Islands (FR : South Georgia & the South Sandwich Islands)
→ Correspondances par inclusion :
  [0] Georgia


Sélectionner le numéro de la correspondance à valider (Entrée pour ignorer) :  


❌ Aucune correspondance validée pour South Georgia & the South Sandwich Islands

🌍 Nom : Bosnia & Herzegovina (FR : Bosnie-Herzégovine)
→ Correspondances floues potentielles dans df_merged['Country'] :
  [0] Bosnia and Herzegovina


Sélectionner le numéro de la correspondance à valider (Entrée pour ignorer) :  0


✅ Correspondance validée (floue) : Bosnia & Herzegovina → Bosnia and Herzegovina

✅ Mise à jour terminée.


In [64]:
# Sauvegarde du GeoDataFrame mis à jour en GeoJSON
filepath= base_path/ "Data" / 'SIG' / "world.geojson"
gdf_world = gdf_world.rename(columns={'iso_3166_1_alpha_2_codes': 'Country_code'})

gdf_world.to_file(filepath, driver="GeoJSON", encoding="utf-8")

print(f"✅ Fichier enregistré : {filepath}")


✅ Fichier enregistré : C:\Users\Aubin\Documents\NetZero\Data\SIG\world.geojson


### GO

In [128]:
# Fusionner avec df_merged
df_merged = df_long.merge(
    gdf_world[['Country_code', 'Country']],
    on='Country',
    how='left'
)

# Vérifier que la colonne existe
if 'Country_code' not in df_merged.columns:
    df_merged['Country_code'] = pd.NA

print("✅ Fusion terminée.")
print(f"Lignes fusionnées : {len(df_merged)}")
print(f"Lignes sans correspondance de code pays AVANT mapping manuel : {df_merged['Country_code'].isna().sum()}")

# --- 🔧 Ajout du mapping manuel pour certains pays ---
manual_mapping = {
    "Laos": "LA",
    "Lao People's Democratic Republic": "LA",
    "North Korea": "KP",
    "Democratic People's Republic of Korea": "KP",
    "North Macedonia": "MK",
    "Macedonia": "MK",
    "Occupied Palestinian Territory": "PS",
    "State of Palestine": "PS",
    "South Korea": "KR",
    "Republic of Korea": "KR",
    "United Kingdom": "GB",
    "United Kingdom of Great Britain and Northern Ireland": "GB",
    "Namibia": "NAM"  # ton code personnalisé
}

# Appliquer les codes manuels
df_merged['Country_code'] = df_merged.apply(
    lambda row: manual_mapping.get(row['Country'], row['Country_code']),
    axis=1
)

print(f"Lignes sans correspondance de code pays APRÈS mapping manuel : {df_merged['Country_code'].isna().sum()}")


✅ Fusion terminée.
Lignes fusionnées : 40194
Lignes sans correspondance de code pays AVANT mapping manuel : 8526
Lignes sans correspondance de code pays APRÈS mapping manuel : 7482


In [121]:
#enregistrer dans data_intermediate
save_long_dataframe(
    df_merged,
    base_path,
    folder='data_intermediate',
    indicator=indicator,
    source=source
)


DataFrame sauvegardé dans C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Deforestation_GCB.csv


WindowsPath('C:/Users/Aubin/Documents/NetZero/Data/data_intermediate/LULUCF_Deforestation_GCB.csv')

In [129]:
# Filtrer les pays sans correspondance
countries_without_match = df_merged[df_merged['Country_code'].isna()]['Country'].dropna().drop_duplicates().sort_values()

# Afficher le résultat
print(f"🌍 Nombre de pays sans correspondance : {len(countries_without_match)}")
display(countries_without_match.to_frame().reset_index(drop=True))


🌍 Nombre de pays sans correspondance : 43


,Country
0,Africa
1,Anguilla
2,Aruba
3,Asia
4,Bermuda
5,"Bonaire, Saint Eustatius and Saba"
6,British Virgin Islands
7,Bunkers
8,Central America
9,Curaçao


In [130]:
import pandas as pd

def manual_country_code_match(df_merged, gdf_world, country_codes_path=r'C:\Users\Aubin\Documents\NetZero\Data\SIG\country_codes_clean.csv'):
    """
    Complète manuellement ou automatiquement les codes ISO (Country_code) pour les pays
    de df_merged qui n'ont pas de correspondance dans gdf_world['Country_code'].
    """
    
    # --- 🔧 Mapping manuel forcé (prioritaire) ---
    manual_mapping = {
        "Laos": "LA",
        "Lao People's Democratic Republic": "LA",
        "North Korea": "KP",
        "Democratic People's Republic of Korea": "KP",
        "North Macedonia": "MK",
        "Macedonia": "MK",
        "Occupied Palestinian Territory": "PS",
        "State of Palestine": "PS",
        "South Korea": "KR",
        "Republic of Korea": "KR",
        "United Kingdom": "GB",
        "United Kingdom of Great Britain and Northern Ireland": "GB",
        "Namibia": "NAM"   # (attention : ce n’est pas le code ISO, mais ce que tu veux)
    }

    # Charger le fichier CSV des codes ISO
    country_codes_df = pd.read_csv(country_codes_path)

    # Mapping existant depuis gdf_world
    existing_mapping = gdf_world.set_index('Country')['Country_code'].to_dict()

    # Mapping depuis CSV
    csv_mapping = country_codes_df.set_index('Country')['Alpha-2 code'].to_dict()

    # Identifier les pays sans code
    unmatched_countries = (
        df_merged.loc[df_merged['Country_code'].isna(), 'Country']
        .dropna()
        .unique()
    )

    print(f"\n🔎 {len(unmatched_countries)} pays sans Country_code.\n")

    for c in unmatched_countries:
        print(f"\n🌍 Pays : {c}")

        # --- 1️⃣ Vérifier le mapping manuel ---
        if c in manual_mapping:
            saisie = manual_mapping[c]
            print(f"→ Code trouvé dans le mapping manuel : {saisie}")

        # --- 2️⃣ Vérifier le CSV ---
        elif c in csv_mapping:
            saisie = csv_mapping[c]
            print(f"→ Code trouvé automatiquement depuis CSV : {saisie}")

        # --- 3️⃣ Chercher dans gdf_world (suggestions partielles) ---
        else:
            suggestions = gdf_world[gdf_world['Country'].str.contains(c, case=False, na=False)]

            if not suggestions.empty:
                print("→ Suggestions depuis gdf_world :")
                for i, row in suggestions.iterrows():
                    print(f"  [{i}] {row['Country']} → {row['Country_code']}")

            # Saisie manuelle
            saisie = input("Entrer le Country_code ISO correspondant (ou Entrée pour ignorer) : ").strip().upper()
            if saisie == "":
                print("⏭️ Ignoré.")
                continue

        # Appliquer la saisie
        df_merged.loc[df_merged['Country'] == c, 'Country_code'] = saisie
        print(f"✅ Country_code ajouté : {c} → {saisie}")

    print("\n✅ Saisie automatique et manuelle des codes terminée.")
    return df_merged


In [132]:
df_merged=manual_country_code_match(df_merged, gdf_world)


🔎 43 pays sans Country_code.


🌍 Pays : Anguilla
→ Code trouvé automatiquement depuis CSV : AI
✅ Country_code ajouté : Anguilla → AI

🌍 Pays : Aruba
→ Code trouvé automatiquement depuis CSV : AW
✅ Country_code ajouté : Aruba → AW

🌍 Pays : Bermuda
→ Code trouvé automatiquement depuis CSV : BM
✅ Country_code ajouté : Bermuda → BM

🌍 Pays : Bonaire, Saint Eustatius and Saba


KeyboardInterrupt: Interrupted by user

## DF FINAL

In [109]:
#enregistrer dans data_intermediate
save_long_dataframe(
    df_merged,
    base_path,
    folder='data_intermediate',
    indicator=indicator,
    source=source
)


DataFrame sauvegardé dans C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Forest_Regrowth_GCB.csv


WindowsPath('C:/Users/Aubin/Documents/NetZero/Data/data_intermediate/LULUCF_Forest_Regrowth_GCB.csv')

In [90]:
df_merged

,Year,Country,Value,Unit,Indicator,Source,Country_code
0,1960,Afghanistan,3472860.0,tC,LULUCF Other Emissions,GCB,AF
1,1961,Afghanistan,2936880.0,tC,LULUCF Other Emissions,GCB,AF
2,1962,Afghanistan,2676090.0,tC,LULUCF Other Emissions,GCB,AF
3,1963,Afghanistan,2581340.0,tC,LULUCF Other Emissions,GCB,AF
4,1964,Afghanistan,2136740.0,tC,LULUCF Other Emissions,GCB,AF
...,...,...,...,...,...,...,...
12670,2020,Zimbabwe,-1382330.0,tC,LULUCF Other Emissions,GCB,ZW
12671,2021,Zimbabwe,-1024560.0,tC,LULUCF Other Emissions,GCB,ZW
12672,2022,Zimbabwe,-1280580.0,tC,LULUCF Other Emissions,GCB,ZW
12673,2023,Zimbabwe,-1315430.0,tC,LULUCF Other Emissions,GCB,ZW


###  CHANGE UNIT

In [71]:
def browse_and_convert_units(
        base_path,
        folder_in="data_intermediate",
        value_col="Value",
        unit_col="Unit",
        sep=",",
        encoding="utf-8"
    ):
    """
    Parcourt tous les fichiers CSV du dossier, demande si on veut changer l'unité
    et applique la conversion directement dans chaque fichier.

    Paramètres :
    - base_path : Path de base
    - folder_in : dossier contenant les CSV
    - value_col : colonne des valeurs
    - unit_col : colonne de l'unité
    """

    import pandas as pd
    from pathlib import Path
    import glob

    # Construire le chemin des fichiers
    folder_path = base_path / "Data" / folder_in
    files = glob.glob(str(folder_path / "*.csv"))

    if not files:
        print(f"Aucun fichier CSV trouvé dans {folder_path}")
        return None

    print(f"🔍 {len(files)} fichiers trouvés dans {folder_path}\n")

    for file in files:
        print("═══════════════════════════════════════════════")
        print(f"📄 Fichier : {file}")

        # Charger le fichier
        df = pd.read_csv(file, sep=sep, encoding=encoding)

        # Vérification des colonnes
        if value_col not in df.columns or unit_col not in df.columns:
            print(f"⚠️  Fichier ignoré (colonnes manquantes '{value_col}' ou '{unit_col}')\n")
            continue

        # Unité actuelle
        current_unit = df[unit_col].iloc[0]
        print(f"🔍 Unité actuelle : {current_unit}")

        # Demander si on veut convertir
        resp = input("👉 Voulez-vous changer l'unité ? (o/n) : ").strip().lower()
        if resp != "o":
            print("➡ Aucun changement pour ce fichier.\n")
            continue

        # Facteur multiplicatif
        while True:
            try:
                factor = float(input("➡ Facteur multiplicatif : "))
                break
            except ValueError:
                print("❌ Entrer un nombre valide.")

        # Nouvelle unité
        new_unit = input("➡ Nouvelle unité : ").strip()
        if new_unit == "":
            print("❌ Unité vide — fichier non modifié.\n")
            continue

        # Appliquer la conversion
        print(f"🔄 Conversion : {value_col} × {factor}")
        df[value_col] = df[value_col] * factor
        df[unit_col] = new_unit

        # Sauvegarder
        df.to_csv(file, sep=sep, index=False, encoding=encoding)
        print(f"💾 Fichier mis à jour : {file}\n")

    print("🎉 Conversion terminée pour tous les fichiers.")


In [73]:
from pathlib import Path

browse_and_convert_units(base_path, folder_in="data_intermediate")


🔍 2 fichiers trouvés dans C:\Users\Aubin\Documents\NetZero\Data\data_intermediate

═══════════════════════════════════════════════
📄 Fichier : C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\Fossil_CO2_emissions_GCB.csv
🔍 Unité actuelle : tC


👉 Voulez-vous changer l'unité ? (o/n) :  n


➡ Aucun changement pour ce fichier.

═══════════════════════════════════════════════
📄 Fichier : C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Net_emissions_GCB.csv
🔍 Unité actuelle : MtC/year


👉 Voulez-vous changer l'unité ? (o/n) :  o
➡ Facteur multiplicatif :  1000000
➡ Nouvelle unité :  tC


🔄 Conversion : Value × 1000000.0
💾 Fichier mis à jour : C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Net_emissions_GCB.csv

🎉 Conversion terminée pour tous les fichiers.


### CONCAT

In [12]:
def concat_intermediate_files(base_path, folder_in="data_intermediate", folder_out="data_final", final_filename="data_final.csv", sep=',', encoding='utf-8'):
    """
    Concatène tous les fichiers CSV dans un dossier intermédiaire et sauvegarde le résultat final.
    Affiche les fichiers trouvés et le nombre de lignes par fichier.

    Paramètres :
    - folder_in : dossier contenant les fichiers CSV intermédiaires
    - folder_out : dossier où sauvegarder le fichier final
    - final_filename : nom du fichier CSV final
    - sep : séparateur CSV
    - encoding : encodage du fichier
    """
    import pandas as pd
    from pathlib import Path
    import glob

    # Construire le chemin des fichiers
    folder_path = base_path / "Data" / folder_in
    files = glob.glob(str(folder_path / "*.csv"))

    if not files:
        print(f"Aucun fichier CSV trouvé dans {folder_path}")
        return None

    print("Fichiers trouvés et nombre de lignes :")
    dfs = []
    for f in files:
        df = pd.read_csv(f, sep=sep, encoding=encoding)
        print(f"{f} colonnes : {df.columns.tolist()}")
        print(f"- {f} : {len(df)} lignes")
        dfs.append(df)

    # Concaténer tous les fichiers
    df_final = pd.concat(dfs, ignore_index=True)
    print(f"{len(files)} fichiers fusionnés pour obtenir {len(df_final)} lignes au total.")

    # Sauvegarder le résultat final
    save_path = base_path / "Data" / folder_out / final_filename
    save_path.parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(save_path, sep=sep, index=False, encoding=encoding)

    print(f"DataFrame final sauvegardé dans {save_path}")
    return df_final


In [13]:
#Concaténer toutes les bases de données dans data_intermediate et enregistrer dans data_final
df_final = concat_intermediate_files(
    base_path,
    folder_in="data_intermediate",
    folder_out="data_final",
    final_filename="data_final_all.csv"
)
df_final.head()


Fichiers trouvés et nombre de lignes :
C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\Fossil_CO2_emissions_GCB.csv colonnes : ['Year', 'Country', 'Value', 'Unit', 'Indicator', 'Source', 'Country_code']
- C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\Fossil_CO2_emissions_GCB.csv : 24636 lignes
C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Net_emissions_GCB.csv colonnes : ['Year', 'Country', 'Value', 'Unit', 'Indicator', 'Source', 'Country_code']
- C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Net_emissions_GCB.csv : 12480 lignes
2 fichiers fusionnés pour obtenir 37116 lignes au total.
DataFrame final sauvegardé dans C:\Users\Aubin\Documents\NetZero\Data\data_final\data_final_all.csv


,Year,Country,Value,Unit,Indicator,Source,Country_code
0,1949,Afghanistan,4000.0,tC,Fossil CO2 emissions,GCB,AF
1,1950,Afghanistan,23000.0,tC,Fossil CO2 emissions,GCB,AF
2,1951,Afghanistan,25000.0,tC,Fossil CO2 emissions,GCB,AF
3,1952,Afghanistan,25000.0,tC,Fossil CO2 emissions,GCB,AF
4,1953,Afghanistan,29000.0,tC,Fossil CO2 emissions,GCB,AF


In [133]:
def concat_and_add_cumulative(base_path,
                              folder_in="data_intermediate",
                              folder_out="data_final",
                              final_filename="data_final.csv",
                              sep=",",
                              encoding="utf-8",
                              group_cols=["Country_code"]):
    """
    Parcourt un dossier contenant des fichiers CSV, ajoute Type='Annual',
    puis crée une version Cumulative en cumulant la colonne 'Value' sur 'Year',
    en respectant le pays (Code_country).

    Paramètres :
    - group_cols : colonnes pour grouper les valeurs afin de cumuler (inclut Code_country).
    """

    import pandas as pd
    from pathlib import Path
    import glob

    folder_path = base_path / "Data" / folder_in
    files = glob.glob(str(folder_path / "*.csv"))

    if not files:
        print(f"Aucun fichier CSV trouvé dans {folder_path}")
        return None

    dfs_all = []

    print("Fichiers trouvés :")
    for f in files:
        print(f"→ {f}")

        df = pd.read_csv(f, sep=sep, encoding=encoding)

        # Vérifications
        required_cols = ["Year", "Value", "Country_code"]
        for col in required_cols:
            if col not in df.columns:
                print(f"⚠️ Fichier ignoré : colonne manquante '{col}' → {f}")
                break
        else:
            # ANNUAL
            df_annual = df.copy()
            df_annual["Type"] = "Annual"

            # CUMULATIVE
            df_cum = df.copy()
            df_cum["Type"] = "Cumulative"

            # Cumul par Code_country + autres group_cols
            df_cum["Value"] = (
                df_cum
                .sort_values("Year")
                .groupby(group_cols)["Value"]
                .cumsum()
            )

            # Combine Annual + Cumulative
            dfs_all.append(pd.concat([df_annual, df_cum], ignore_index=True))

    # Fusion de tous les fichiers
    df_final = pd.concat(dfs_all, ignore_index=True)

    # Sauvegarde
    save_path = base_path / "Data" / folder_out / final_filename
    save_path.parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(save_path, sep=sep, index=False, encoding=encoding)

    print(f"\n✔ Données fusionnées et sauvegardées dans : {save_path}")
    print(f"✔ Total : {len(df_final):,} lignes")

    return df_final


In [134]:
df_final=concat_and_add_cumulative(base_path,
                              folder_in="data_intermediate",
                              folder_out="data_final",
                              final_filename="data_final_all.csv",
                              sep=",",
                              encoding="utf-8",
                              group_cols=["Country_code"])

Fichiers trouvés :
→ C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Deforestation_GCB.csv
→ C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Forest_Regrowth_GCB.csv
→ C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Net_emissions_GCB.csv
→ C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Other_Emissions_GCB.csv
→ C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Peat_Emissions_GCB.csv
→ C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Wood_Harvesting_GCB.csv

✔ Données fusionnées et sauvegardées dans : C:\Users\Aubin\Documents\NetZero\Data\data_final\data_final_all.csv
✔ Total : 151,710 lignes


In [52]:
from pathlib import Path
import pandas as pd

folder = base_path / 'Data' / 'data_intermediate'

# Loop over all files in the folder (e.g., CSV files)
for filepath in folder.glob('*.csv'):  # adjust pattern if needed
     # read the file
    df_fixed = fix_missing_country_codes(filepath)  # apply your function



Loading data from: C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\Fossil_CO2_emissions_GCB.csv
File loaded: C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\Fossil_CO2_emissions_GCB.csv
------------------------------------------------------------
Countries missing Country_code:
 - Namibia

Please enter the missing Country_code values.



Enter Country_code for 'Namibia':  NAM



Updated file saved!
 → C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\Fossil_CO2_emissions_GCB.csv
Loading data from: C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Net_emissions_GCB.csv
File loaded: C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Net_emissions_GCB.csv
------------------------------------------------------------
Countries missing Country_code:
 - Namibia

Please enter the missing Country_code values.



Enter Country_code for 'Namibia':  NAM



Updated file saved!
 → C:\Users\Aubin\Documents\NetZero\Data\data_intermediate\LULUCF_Net_emissions_GCB.csv


In [ ]:
### CHECK NO COUNTRY CODE BUT DATA AND UPDATE

In [48]:
import pandas as pd
def import_csv_data(filepath):
    print("Loading data from:", filepath.resolve())  # debug
    df = pd.read_csv(filepath)
    return df
    
def fix_missing_country_codes(filepath, country_col="Country", code_col="Country_code"):
    """
    Load a file, detect countries missing country_code, ask user for the code,
    update the file, and save the corrected version.
    """
    # Load file
    df = import_csv_data(filepath)
    print(f"File loaded: {filepath}")
    print("-" * 60)

    # Identify missing codes
    missing_mask = df[code_col].isna() | (df[code_col].astype(str).str.strip() == "")
    missing_countries = df.loc[missing_mask, country_col].unique()

    if len(missing_countries) == 0:
        print("No missing Country_code found. Nothing to fix.")
        return df

    print("Countries missing Country_code:")
    for c in missing_countries:
        print(f" - {c}")

    print("\nPlease enter the missing Country_code values.\n")

    # Ask manually for each missing country
    replacement_map = {}
    for country in missing_countries:
        code = input(f"Enter Country_code for '{country}': ").strip()
        replacement_map[country] = code

    # Apply replacements
    for country, code in replacement_map.items():
        df.loc[df[country_col] == country, code_col] = code

    # Save file (overwrite)
    df.to_csv(filepath, index=False)
    print("\nUpdated file saved!")
    print(f" → {filepath}")

    return df




Loading data from: C:\Users\Aubin\Documents\NetZero\Data\data_final\data_final_all_norm.csv
File loaded: C:\Users\Aubin\Documents\NetZero\Data\data_final\data_final_all_norm.csv
------------------------------------------------------------
No missing Country_code found. Nothing to fix.
